In [ ]:
# Visualization 2: Is there a regional effect in the data? Are some names more popular in some regions? Are popular names generally popular across the whole country?

In [16]:
import pandas as pd
import altair as alt
import json
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import HTML

In [12]:
# Chargement des données
df = pd.read_csv('dpt2020.csv', sep=';')
df = df[df['preusuel'] != '_PRENOMS_RARES']
df = df[df['annais'] != 'XXXX']
df['annais'] = df['annais'].astype(int)

# Chargement du GeoJSON
with open("departements.geojson", "r", encoding="utf-8") as f:
    geojson = json.load(f)

departements_geo = alt.Data(
    values=geojson['features'],
    format={'type': 'json', 'property': 'features'}
)

In [ ]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
from IPython.display import display
import json

# Chargement des données
df = pd.read_csv('dpt2020.csv', sep=';')
df = df[df['preusuel'] != '_PRENOMS_RARES']
df = df[df['annais'] != 'XXXX']
df['annais'] = df['annais'].astype(int)

# Slider
annee_min = df['annais'].min()
annee_max = df['annais'].max()

def afficher_carte(year):
    dominants = (
        df[df['annais'] == year]
        .groupby(['dpt', 'preusuel'])['nombre']
        .sum()
        .reset_index()
        .sort_values('nombre', ascending=False)
        .drop_duplicates('dpt')
    )

    geo_data = alt.Data(url='departements.geojson', format={'type': 'json', 'property': 'features'})

    chart = alt.Chart(geo_data).mark_geoshape(
        stroke='lightgray'
    ).transform_lookup(
        lookup='properties.code',
        from_=alt.LookupData(dominants, 'dpt', ['preusuel', 'nombre'])
    ).encode(
        color=alt.Color('preusuel:N', title='Prénom dominant'),
        tooltip=[
            alt.Tooltip('properties.nom:N', title='Département'),
            alt.Tooltip('preusuel:N', title='Prénom'),
            alt.Tooltip('nombre:Q', title='Nombre')
        ]
    ).project('mercator').properties(
        width=700,
        height=600,
        title=f'Prénom dominant par département en {year}'
    )

    return chart

widgets.interact(afficher_carte, year=widgets.IntSlider(value=2020, min=annee_min, max=annee_max, step=1))


interactive(children=(IntSlider(value=2020, description='year', max=2020, min=1900), Output()), _dom_classes=(…

<function __main__.afficher_carte(year)>